In [2]:
import numpy as np
from PIL import Image, ImageTk, ImageFilter
import tkinter as tk
from tkinter import filedialog

def carregar_imagem():
    """Função para carregar imagem e exibir na interface."""
    global img_original, img_exibicao
    caminho_imagem = filedialog.askopenfilename()
    if caminho_imagem:
        img_original = Image.open(caminho_imagem).convert('L')
        img_exibicao = ImageTk.PhotoImage(img_original)
        painel_imagem_original.config(image=img_exibicao)
        painel_imagem_original.image = img_exibicao

def filtro_passa_baixa():
    """Aplica filtro passa-baixa (Gaussian Blur) e exibe imagem filtrada."""
    if img_original is not None:
        img_filtrada = img_original.filter(ImageFilter.GaussianBlur(radius=7))
        exibir_imagem_filtrada(img_filtrada)

def filtro_media():
    """Aplica filtro passa-baixa (média) manualmente e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        kernel_size = 15
        kernel = np.ones((kernel_size, kernel_size)) / (kernel_size ** 2)
        img_filtrada = aplicar_filtro_movel(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_filtrada.astype(np.uint8))
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_mediana():
    """Aplica filtro passa-baixa (mediana) manualmente e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        kernel_size = 15
        img_filtrada = aplicar_filtro_mediana(img_array, kernel_size)
        img_filtrada_pil = Image.fromarray(img_filtrada.astype(np.uint8))
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_passa_alta():
    """Aplica filtro passa-alta (subtração de passa-baixa) e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        kernel_size = 5
        kernel = np.ones((kernel_size, kernel_size)) / (kernel_size ** 2)
        img_suave = aplicar_filtro_movel(img_array, kernel)
        img_filtrada = img_array.astype(np.float32) - img_suave.astype(np.float32)
        img_filtrada = np.clip(2.0 * img_filtrada, 0, 255).astype(np.uint8)
        img_filtrada_pil = Image.fromarray(img_filtrada)
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_laplaciano():
    """Aplica filtro passa-alta (Laplaciano) e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        laplacian_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
        img_filtrada = aplicar_filtro_movel(img_array, laplacian_kernel)
        img_filtrada = np.clip(img_filtrada, 0, 255).astype(np.uint8)
        img_filtrada_pil = Image.fromarray(img_filtrada)
        exibir_imagem_filtrada(img_filtrada_pil)

def filtro_sobel():
    """Aplica filtro Sobel (detecção de bordas) e exibe imagem filtrada."""
    if img_original is not None:
        img_array = np.array(img_original)
        sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
        grad_x = aplicar_filtro_movel(img_array, sobel_x)
        grad_y = aplicar_filtro_movel(img_array, sobel_y)
        img_filtrada = np.hypot(grad_x, grad_y)
        img_filtrada = (img_filtrada / img_filtrada.max() * 255).astype(np.uint8)
        img_filtrada_pil = Image.fromarray(img_filtrada)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_filtro_movel(imagem, kernel):
    """Função genérica para aplicar a convolução com o kernel fornecido."""
    h, w = imagem.shape
    kh, kw = kernel.shape
    img_filtrada = np.zeros_like(imagem)
    for i in range(kh // 2, h - kh // 2):
        for j in range(kw // 2, w - kw // 2):
            img_filtrada[i, j] = np.sum(imagem[i - kh // 2:i + kh // 2 + 1, j - kw // 2:j + kw // 2 + 1] * kernel)
    return img_filtrada

def aplicar_filtro_mediana(imagem, kernel_size):
    """Aplica filtro de mediana manualmente."""
    h, w = imagem.shape
    img_filtrada = np.zeros_like(imagem)
    for i in range(kernel_size // 2, h - kernel_size // 2):
        for j in range(kernel_size // 2, w - kernel_size // 2):
            patch = imagem[i - kernel_size // 2:i + kernel_size // 2 + 1, j - kernel_size // 2:j + kernel_size // 2 + 1]
            img_filtrada[i, j] = np.median(patch)
    return img_filtrada

# Funções para Operações Morfológicas
def aplicar_erosao():
    if img_original is not None:
        img_array = np.array(img_original)
        kernel = np.ones((5, 5), np.uint8)
        img_erodida = erosao(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_erodida)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_dilatacao():
    if img_original is not None:
        img_array = np.array(img_original)
        kernel = np.ones((5, 5), np.uint8)
        img_dilatada = dilatacao(img_array, kernel)
        img_filtrada_pil = Image.fromarray(img_dilatada)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_abertura():
    if img_original is not None:
        img_array = np.array(img_original)
        kernel = np.ones((5, 5), np.uint8)
        img_aberta = dilatacao(erosao(img_array, kernel), kernel)
        img_filtrada_pil = Image.fromarray(img_aberta)
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_fechamento():
    if img_original is not None:
        img_array = np.array(img_original)
        kernel = np.ones((5, 5), np.uint8)
        img_fechada = erosao(dilatacao(img_array, kernel), kernel)
        img_filtrada_pil = Image.fromarray(img_fechada)
        exibir_imagem_filtrada(img_filtrada_pil)

# Funções para Segmentação de Imagem
def aplicar_limiarizacao():
    if img_original is not None:
        img_array = np.array(img_original)
        _, img_thresh = np.where(img_array > 128, 255, 0)
        img_filtrada_pil = Image.fromarray(img_thresh.astype(np.uint8))
        exibir_imagem_filtrada(img_filtrada_pil)

def aplicar_limiarizacao_adaptativa():
    if img_original is not None:
        img_array = np.array(img_original)
        img_thresh_adapt = limiarizacao_adaptativa(img_array)
        img_filtrada_pil = Image.fromarray(img_thresh_adapt)
        exibir_imagem_filtrada(img_filtrada_pil)

# Funções de Processamento de Imagem
def erosao(imagem, kernel):
    kh, kw = kernel.shape
    img_erodida = np.zeros_like(imagem)
    for i in range(kh // 2, imagem.shape[0] - kh // 2):
        for j in range(kw // 2, imagem.shape[1] - kw // 2):
            if np.all(imagem[i - kh // 2:i + kh // 2 + 1, j - kw // 2:j + kw // 2 + 1] >= kernel):
                img_erodida[i, j] = 255
    return img_erodida

def dilatacao(imagem, kernel):
    kh, kw = kernel.shape
    img_dilatada = np.zeros_like(imagem)
    for i in range(kh // 2, imagem.shape[0] - kh // 2):
        for j in range(kw // 2, imagem.shape[1] - kw // 2):
            if np.any(imagem[i - kh // 2:i + kh // 2 + 1, j - kw // 2:j + kw // 2 + 1] >= kernel):
                img_dilatada[i, j] = 255
    return img_dilatada

def limiarizacao_adaptativa(imagem, kernel_size=11, C=2):
    img_filtrada = np.zeros_like(imagem)
    pad_size = kernel_size // 2
    imagem_padded = np.pad(imagem, pad_size, mode='reflect')
    for i in range(pad_size, imagem_padded.shape[0] - pad_size):
        for j in range(pad_size, imagem_padded.shape[1] - pad_size):
            local_region = imagem_padded[i - pad_size:i + pad_size + 1, j - pad_size:j + pad_size + 1]
            local_mean = np.mean(local_region)
            img_filtrada[i - pad_size, j - pad_size] = 255 if imagem[i - pad_size, j - pad_size] > local_mean - C else 0
    return img_filtrada


def exibir_imagem_filtrada(img_filtrada):
    """Exibe a imagem filtrada na interface."""
    img_exibicao_filtrada = ImageTk.PhotoImage(img_filtrada)
    painel_imagem_filtrada.config(image=img_exibicao_filtrada)
    painel_imagem_filtrada.image = img_exibicao_filtrada

def criar_interface():
    """Configura a interface gráfica com botões e painel de exibição."""
    global janela, painel_imagem_original, painel_imagem_filtrada
    janela = tk.Tk()
    janela.title("Editor de Fotos - Filtros de Imagem")
    painel_imagem_original = tk.Label(janela)
    painel_imagem_original.grid(row=0, column=0)
    painel_imagem_filtrada = tk.Label(janela)
    painel_imagem_filtrada.grid(row=0, column=1)
    criar_botoes()
    janela.mainloop()

def criar_botoes():
    """Cria os botões para carregar imagem e aplicar filtros."""
    botao_carregar = tk.Button(janela, text="Carregar Imagem", command=carregar_imagem)
    botao_carregar.grid(row=1, column=0)
    botao_filtro_passa_baixa = tk.Button(janela, text="Filtro Passa-Baixa (Gaussian)", command=filtro_passa_baixa)
    botao_filtro_passa_baixa.grid(row=1, column=1)
    botao_filtro_media = tk.Button(janela, text="Filtro Passa-Baixa (Média)", command=filtro_media)
    botao_filtro_media.grid(row=1, column=2)
    botao_filtro_mediana = tk.Button(janela, text="Filtro Passa-Baixa (Mediana)", command=filtro_mediana)
    botao_filtro_mediana.grid(row=1, column=3)
    botao_filtro_passa_alta = tk.Button(janela, text="Filtro Passa-Alta (Subtração de Passa-Baixa)", command=filtro_passa_alta)
    botao_filtro_passa_alta.grid(row=2, column=1)
    botao_filtro_laplaciano = tk.Button(janela, text="Filtro Passa-Alta (Laplaciano)", command=filtro_laplaciano)
    botao_filtro_laplaciano.grid(row=2, column=2)
    botao_filtro_sobel = tk.Button(janela, text="Filtro Passa-Alta (Sobel)", command=filtro_sobel)
    botao_filtro_sobel.grid(row=2, column=3)

    # Operações Morfológicas
    botao_erosao = tk.Button(janela, text="Erosão", command=aplicar_erosao)
    botao_erosao.grid(row=2, column=0)

    botao_dilatacao = tk.Button(janela, text="Dilatação", command=aplicar_dilatacao)
    botao_dilatacao.grid(row=2, column=1)

    botao_abertura = tk.Button(janela, text="Abertura", command=aplicar_abertura)
    botao_abertura.grid(row=3, column=0)

    botao_fechamento = tk.Button(janela, text="Fechamento", command=aplicar_fechamento)
    botao_fechamento.grid(row=3, column=1)

    # Segmentação de Imagem
    botao_limiarizacao = tk.Button(janela, text="Limiarização", command=aplicar_limiarizacao)
    botao_limiarizacao.grid(row=4, column=0)

    botao_limiarizacao_adaptativa = tk.Button(janela, text="Limiarização Adaptativa", command=aplicar_limiarizacao_adaptativa)
    botao_limiarizacao_adaptativa.grid(row=4, column=1)

# Executa o programa
if __name__ == "__main__":
    criar_interface()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/tmp/ipykernel_35390/2054897893.py", line 133, in aplicar_limiarizacao
    _, img_thresh = np.where(img_array > 128, 255, 0)
ValueError: too many values to unpack (expected 2)
